# Faster Question-answering on IPU using Packed BERT - Fine-tuning and Inference

This notebook describes how to fine-tune BERT from [🤗 Transformers](https://github.com/huggingface/transformers) for question answering using the SQuAD(v1) dataset using [packing](https://towardsdatascience.com/introducing-packed-bert-for-2x-faster-training-in-natural-language-processing-eadb749962b1), an optimisation method originally used for 2x faster BERT pre-training, which can now also provide massive throughput increases for fine-tuning and batched inference! 

**So, what *is* packing?** The basic idea of "packing" a dataset is to utilise the requirement for constant-shaped inputs into a model. Instead of padding it with empty, unused space, we can recycle this unused space and fill it with more inputs! The architecture of transformer models like BERT supports this, and lets us optimally use this space to process multiple sequences within one input.

**And here is why you might want to use it:** Having a single input that contains multiple sequences leads to multiple sequences being processed in parallel in a single pass within a single iteration inside a batch, increasing the "effective" batch size of the model by a considerable factor in many cases, and most importantly, increasing model throughput for training and batched inference significantly.

This notebook builds on the process described in the notebook on "fine-tuning a model on a question-answering task" `natural-language-processing/other-use-cases/question_answering.ipynb`. In this notebook, we will how to adapt the the `BertForQuestionAnswering` model to accommodate a packed dataset. 

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Natural language processing | Question answering | PackedBERT | SQuADv1| Inference | |   |

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies and configuration

In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

Install the dependencies for this notebook.

In [ ]:
%pip install optimum-graphcore==0.7
%pip install scikit-learn "datasets>=2.7.0" evaluate tokenizers matplotlib scipy huggingface_hub

%pip install graphcore-cloud-tools[logger]@git+https://github.com/graphcore/graphcore-cloud-tools
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

n_ipu = int(os.getenv("NUM_AVAILABLE_IPU", 4))
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache/") + "/packed_bert_squad/"

In this notebook, we use both data parallelism and pipeline parallelism (see this [tutorial on efficient data loading](https://github.com/graphcore/examples/blob/master/tutorials/tutorials/pytorch/efficient_data_loading/walkthrough.ipynb) for more information), so the "global" batch size, which is the number of data elements passed for one gradient calculation on the IPU, is calculated using the number of device iterations (`device_iterations`), the number of gradient accumulation steps (`gradient_accumulation_steps`), the replication factor (`replication_factor`) and the maximum number of sequences in a pack (`max_seq_per_pack`) for training, such that:

```
global_training_batch_size = micro_batch_size * device_iterations * gradient_accumulation_steps * replication_factor
```

Note: we define a "micro" batch size, which is the local batch size that would be passed into the model on the CPU.

Depending on your model and the IPU Pod machine you are using, you might need to adjust these three batch-size-related arguments.

`max_seq_per_pack` highlights the benefit of packing multiple sequences into one input sequence given there is enough space for them. It shows that multiple sequences are processed effectively in parallel within the model, using up space that would essentially be padding if one sequence were passed at a time. This is a much more efficient way to send inputs into the model, and improves the global batch size to a best-case-scenario of:

```
global_training_batch_size = micro_batch_size * device_iterations * gradient_accumulation_steps * replication_factor * max_seq_per_pack
```

Realistically, the global batch size will not always be multiplied by the *maximum* number of sequences in a packed sequence, but rather the *average* number of sequences in a packed sequence, and will depend on the sequence length distribution within any given dataset.

In [ ]:
model_checkpoint="bert-base-uncased" # Default uncased pre-trained BERT checkpoint
ipu_config_name="Graphcore/bert-base-uncased" # Default Graphcore IPU config initialisation for pre-trained BERT
max_seq_length=512 # The maximum sequence length allowed for sequences in the model.
number_ipus_per_replica = 4
number_of_replicas = n_ipu // number_ipus_per_replica
# The size of the machine and the length of the pipeline impact the number of 
# samples that need to be processed between gradient updates
gradient_accumulation_steps = 32 // number_of_replicas
device_iterations = 32
micro_batch_size=2
model_task="squad" 

Gradients are not calculated during validation, so gradient accumulation is not applicable, and the global batch size for validation can be defined separately as:

```
global_validation_batch_size=micro_batch_size*device_iterations*replication_factor*max_seq_per_pack
```

In Graphcore Optimum, we can define inference-specific values for `device iterations` and `replication factor`, which can be adjusted to create larger batches to compensate for the lack of a gradient accumulation factor.

## Loading the dataset

The next step is to use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the dataset from the 🤗 Hub, and to use the [🤗 Evaluate](https://github.com/huggingface/evaluate) library to load the evaluation metrics for the SQuAD model. This will allow easy performance metric analysis during validation.

In [ ]:
from datasets import load_dataset, load_metric
import evaluate

dataset = load_dataset(model_task) # Load dataset
metric = evaluate.load(model_task) # Load metric for dataset

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key each for the training, validation and test sets:

In [ ]:
dataset

To access an actual element, you need to select a split ("train" in the example) and then specify an index:

In [ ]:
dataset["train"][0]

In the SQuAD dataset, we have a `question`, its `context` which is an excerpt of text which includes the answer as well as surrounding context, and the `answer` key, which holds the start position of the answer in the context, as well as the answer itself. For a different or custom question-answering dataset, these fields may have different names but serve the same purpose, so pre-defining them is useful.

We  have a configuration describing these necessary keys in the dataset containing the raw data that needs to be pre-processed or tokenised before being passed into the model. These generic keys may change for custom datasets, but the usage of them generally stays the same for a similar fine-tuning task.

In [ ]:
question_key="question"
context_key="context"
answer_key="answers"
train = True
validate = True

## Preprocessing the data

Before we can feed these text samples to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pre-trained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- We get a tokenizer that corresponds to the model architecture we want to use.
- We download the vocabulary used when pre-training this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

The `Dataset` method is also imported, which will allow us to convert our modified and tokenized columns in dictionary form to a dataset.

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset 

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

For SQuAD, we define a custom function to handle the overflows and offset mapping created by generating tokenised inputs from sequences, as well as the start and end positions of the answers which need to be translated from positions of characters to positions of tokens.

The first step is to tokenize the dataset using the tokenizer. Note here that for packing, it is important to **not** pad the dataset, so `padding` should be set to `False`. If we pad, we will have to un-pad when packing sequences into a packed sequence, which is inefficient.

The details of the preprocessing function is outlined in the original (unpacked) question-answering notebook `natural-language-processing/other-use-cases/question_answering.ipynb`. In this case, we can import the preprocessing directly from `utils.packing`, *without* padding for PackedBERT.

In [ ]:
from utils.packing.qa_utils import preprocess_packed_qa

raw_train_dataset = dataset['train']

tokenized_training_dataset = preprocess_packed_qa(
    dataset=raw_train_dataset,
    tokenizer=tokenizer,
    question_key=question_key,
    context_key=context_key,
    answer_key=answer_key,
    sequence_length=max_seq_length,
    padding=False,
    train=True
)


raw_validation_dataset = dataset['validation']

tokenized_validation_dataset = preprocess_packed_qa(
    dataset=raw_validation_dataset,
    tokenizer=tokenizer,
    question_key=question_key,
    context_key=context_key,
    answer_key=answer_key,
    sequence_length=max_seq_length,
    padding=False,
    train=False
)

## Packing the dataset

To implement packing, we need to pack our dataset first. Each new element will be a "pack" containing at most `max_seq_per_pack` sequences.

In [ ]:
max_seq_per_pack = 6

We define the number of labels in our dataset. For SQuAD, this means the number of outputs, which are the positions returned by the model. Since this is not a classification task, `num_labels` is set to 2, to correspond to start and end positions.

We also define the problem type.

In [ ]:
num_labels = 2
problem_type = 'question_answering'

### Packing algorithm

In order to pack efficiently, we will use a histogram-based algorithm. The shortest-pack-first histogram packing (SPFHP) was presented in the Graphcore blog post [introducing Packed BERT for a training speedup in natural language processing](https://www.graphcore.ai/posts/introducing-packed-bert-for-2x-faster-training-in-natural-language-processing). We have adapted the [code](https://github.com/graphcore/tutorials/tree/master/blogs_code/packedBERT) from the blog post for this notebook. The full process of packing the dataset consists of four steps:

1. Create a histogram of the sequence lengths of the dataset.
2. Generate the "strategy" for the dataset using one of the state-of-the-art packing algorithms. The strategy maps out the order and indices of the sequences that need to be packed together.
3. Use this strategy to create the actual dataset, concatenating the tokenized features together for each column in the dataset, including the labels.
4. Finally, pass these new columns into a custom PyTorch dataset, ready to be passed to the PopTorch dataloader!

These steps have been simplified through the easy-to-use `utils.packing` package available in Graphcore Optimum. You can simply generate the packed dataset after the usual tokenization and preprocessing by passing all necessary packing configuration to the `PackedDatasetCreator` class, and generate the ready-to-use PyTorch dataset with `.create()`.

Within the function, there are some column names used by default. The expected default columns for text classification include:
* `input_ids`
* `attention_mask`
* `token_type_ids`
* `labels`

These should all be generated automatically when tokenizing any classification dataset for BERT. However, the labels key, as it is not encoded, may have a different name. For this dataset, the column key for the labels for this dataset is `label`, since the dataset creator expects `labels`, we can pass this to the argument `custom_label_key`, so the class can find our labels. 

The `PackedDatasetCreator` requires different instantiations for different datasets, so it must be called separately for each of our dataset splits. We can set either `training`, `validation` or `inference` to `True` as needed.

In [ ]:
from utils.packing.dataset_creator import PackedDatasetCreator

train_data_packer = PackedDatasetCreator(
    tokenized_dataset = tokenized_training_dataset,
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    training = True,
    num_labels = num_labels,
    problem_type = problem_type,
    algorithm = 'SPFHP'
)

val_data_packer = PackedDatasetCreator(
    tokenized_dataset = tokenized_validation_dataset,
    max_sequence_length = max_seq_length,
    max_sequences_per_pack = max_seq_per_pack,
    validation = True,
    num_labels = num_labels,
    problem_type = problem_type,
    algorithm = 'SPFHP'
)

This will create the strategy and initialise the necessary parameters for packing the dataset. We can see that the ideal speed-up we have achieved is approximately 2.2x the original dataset, which corresponds directly to the average packing factor, which is the average number of sequences within one pack.

The `PackedDatasetCreator` class also has some other features we do not use here for training, such as `pad_to_global_batch_size`. This  feature is useful for performing batched inference on a large set of samples when we do not want to lose any of the samples when creating data iterators using the `poptorch.Dataloader`. In this case, it applies 'vertical' padding to the dataset, adding filler rows to bring the dataset up to a value divisible by the global batch size, and allows for the largest possible batch sizes to be used without any loss of data.

You can also view the histogram generated in the first step of the packing process, to observe whether the distribution of sequence lengths in the dataset will benefit from packing. As a general rule, as long as the average length of the sequences in the dataset is 50% or less of the maximum sequence length, packing will offer at least a 2x throughput benefit, in other words: `throughput_increase ≈ max_seq_len/mean_seq_len`

Many datasets have distributions with much smaller average lengths, and will benefit much more. We can easily observe this distribution by retrieving and plotting the histogram from the data class:

In [ ]:
import matplotlib.pyplot as plt

train_histogram = train_data_packer.histogram

plt.hist(train_histogram, bins = [k for k in range(0,max_seq_length,10)]) 
plt.title("Sequence length histogram") 
plt.xlabel('Sequence lengths')
plt.ylabel('Frequency')
plt.show()

Now we need to create the actual packed dataset (step 3 of the packing process outlined above).

In this stage, we take the strategy for mapping the sequences by size into "packs" that were generated by the packing algorithm, and use this to extract the sequences from the tokenized dataset, inserting them into packs for each column in the dataset. Any remaining space in a pack after the sequences have been concatenated is padded to bring all sequences up to the maximum sequence length.

**Some key features unique to packed datasets are worth mentioning here**:

- The specific attention mask (`attention_mask`) that is generated contains a unique index for each sequence of the pack and `0` for the remaining padding tokens. This, essentially, tells the model where to look from the perspective of a single token, ignoring any encoded information (such as a different sequence) that is not relevant to that token.
    - Example of 3 sequences in a pack: `attention_mask = [1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,0,0,0]`
    - Compared to a single sequence in an unpacked input `attention_mask = [1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0]`
    

- `position_ids` for a pack contain the concatenated `position_ids` of each sequences 
    - For instance given 3 sequences: `[0,1,2,3,4] + [0,1,2,3] + [0,1,2] -> [1,2,3,4,1,2,3,1,2,...,0,0,0]` (note: the CLS tokens position id '0' are also moved the end of the pack)
    
    
- For SQuAD, during training, answers are determined using a start position and end position within the sequence. During preprocessing, these were converted from character positions to token positions. Now, during packing, as tokenized sequences are effectively being concatenated along the same dimension, the positions of the answer will change for any sequence that is not starting at index 0 within a pack. For example, in a pack with 2 sequences:
    - Answer positions before packing:
    ```
    Length of sequence 1: 100 tokens (index 0 to 99)   , start position: 30, end position: 35
    Length of sequence 2: 120 tokens (index 0 to 119)  , start position: 15, end position: 25
    ```
    - Answer positions after packing:
    ```
    Length of sequence 1 in pack 1: 100 tokens (index 0 to 99)   , start position: 30, end position: 35
    Length of sequence 2 in pack 1: 120 tokens (index 100 to 219), start position: 115, end position: 125 
    ```

    - The positions have been shifted by the total length of preceding sequences in the pack. We call this the position offset.


To create a dataloader-ready packed dataset, all you need to do is call the `create()` method:

In [ ]:
packed_train_dataset = train_data_packer.create()
packed_val_dataset = val_data_packer.create()

Let's visualize one sample of the new `packed_train_dataset`:

In [ ]:
packed_train_dataset[133]

## Fine-tuning the model

Now that our data is ready, we can download the pre-trained model and fine-tune it.

### Implement Packed BERT

Some model modifications are required to make packing work with BERT. For SQuAD, we create a custom output class to separate the logits according to each of the sequences within the pack and calculate the loss. The existing class `BertForQuestionAnswering` is extended to `PipelinedPackedBertForQuestionAnswering` which incorporates the required modifications to the model. The crux of these changes is to introduce the new attention mask, and modify the hidden layer output of the model to mask any padded inputs from the logits.

First let's load a default BERT configuration using `AutoConfig`. The config includes a new parameter we must set, `max_sequences_per_pack`. This informs the model of the maximum number of sequences it will need to unpack in the model output. It also allows us to clearly define `num_labels` and `problem_type` for this model.

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)
config.max_sequences_per_pack = max_seq_per_pack
config.num_labels = num_labels

Now we can instantiate the model class with the config, loading the weights from the model checkpoint. For SQuAD, we can determine the number of "labels" as the two output types that will determine whether answers are correct or not, which means the start and end position.

In [ ]:
import torch
import numpy as np
torch.manual_seed(43)
np.random.seed(43)
 
from models.modeling_bert_packed import PipelinedPackedBertForQuestionAnswering

model = PipelinedPackedBertForQuestionAnswering.from_pretrained(model_checkpoint, config=config)

The warning tells us we are throwing away some weights and randomly initializing others. This is absolutely normal in this case, because we are removing the head used to pre-train the model on a masked language modelling objective and replacing it with a new head for question answering, for which we don't have pre-trained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

We can first test the model on a CPU.

In [ ]:
# test the model on CPU
from transformers.data.data_collator import default_data_collator

loader = torch.utils.data.DataLoader(packed_train_dataset,
                             batch_size=2,
                             shuffle=True,
                             drop_last=True,
                             collate_fn=default_data_collator)
data = next(iter(loader))
o = model(**data)
print("Logits shape:", o)

Now, let's prepare the model for an IPU.

First, we set the model in half-precision:

In [ ]:
model.half()

### Define validation metrics for SQuAD

Before training and evaluating, a custom post-processing function needs to be defined for SQuAD. This is because we need to map the predictions of the model back to parts of the context in terms of the character positions in the original untokenized samples. The model predicts logits for the start and end token position of the answer.

The purpose of the function is to identify each of the tokenized features according to `example_ids` and map the start and end token positions for the output, taking the top-*n* logit indices and discarding all invalid solutions. It then uses `offset_mapping` to map the start and end token-level positions back to character-level positions within the context, and generates a text answer using the original context. This text prediction can then be used to calculate accuracy metrics and can be compared to the target answer present in the dataset.

The `postprocess_qa_predictions()` function is adapted for packing, taken directly from the existing [tutorial for fine-tuning SQuAD for the IPU](https://github.com/huggingface/optimum-graphcore/blob/main/notebooks/question_answering.ipynb) for an unpacked dataset. The full description for the use of this function is described in that tutorial. 

The main changes to the function for packing include: 
* Instead of iterating through all the features in the tokenized dataset, and obtaining the `example_id` field created during tokenization of the validation dataset, this function iterates through each feature within each pack, obtaining the corresponding `example_id` value for each feature within the pack. 

* It saves the index of the pack in the dataset, **as well as the index of the feature within the pack**, to allow the function to easily and linearly obtain the features to perform validation on.

This post-processing is available ready-to-use from the packing utils: `utils.packing`, and can simply be initialised.

In [ ]:
from utils.packing.qa_utils import postprocess_packed_qa_predictions

Finally, a `compute_validation_metrics` function is created to take in the post-processed predictions. This obtains the answers from the dataset, maps them according to their `example_id` value to the corresponding prediction, and uses `metric` from the 🤗 Evaluate library to compute the relevant metrics for SQuAD, including an "exact match" accuracy, as well as an F1 score, for each answer. 

In [ ]:
def compute_validation_metrics(predictions, raw_validation_dataset, packed_validation_dataset_unformatted, metric):
    
    target_answers = [
        {"id": ex["id"], "answers": ex["answers"]} for ex in raw_validation_dataset
    ]
    
    final_predictions = postprocess_packed_qa_predictions(
        raw_validation_dataset, packed_validation_dataset_unformatted, predictions
    )

    formatted_predictions = [
        {"id": k, "prediction_text": v} for k, v in final_predictions.items()
    ]

    metrics = metric.compute(predictions=formatted_predictions, references=target_answers)
    
    return metrics


### Train and validate the model using the 🤗 Optimum Graphcore `IPUTrainer`

Now let's prepare the model for the IPU. Instantiate the options and machine configurations and create `IPUTrainer` to efficiently and easily perform training on the IPU in just a few lines.

We need to define `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with a config name or path, which we set earlier. Then we use it to set the mode attribute `model.ipu_config` 

As we are using a pre-trained checkpoint, we can use the existing IPU configuration for `Graphcore/bert-base-uncased` for the custom model. This should require no changes as even though the model has been modified to be compatible with a packed dataset, the pipelining stages and IPU options will remain the same. 

Some of the options have been specified when defining `ipu_config` to highlight the global batch size. This uses the configurations defined at the beginning of this notebook. Note that we can also define inference-specific device iterations and replication factors for performing validation on the model, to modify the validation global batch size.

In [ ]:
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

ipu_config = IPUConfig.from_pretrained(
    ipu_config_name,
    executable_cache_dir = executable_cache_dir,
    gradient_accumulation_steps=gradient_accumulation_steps,
    device_iterations=device_iterations,
    replication_factor=1,
    embedding_serialization_factor=1,
    inference_device_iterations= 64,
    inference_replication_factor=1,
)

To instantiate the `IPUTrainer` class, we will need to define `IPUTrainingArguments`, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model. All other arguments are optional:

In [ ]:
training_args = IPUTrainingArguments(
    output_dir=f"./{model_checkpoint}-{model_task}",
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=9e-05,
    loss_scaling=64.0,
    weight_decay=0.01,
    warmup_ratio=0.25,
    lr_scheduler_type='cosine',
    n_ipu=n_ipu,
    gradient_accumulation_steps=gradient_accumulation_steps,
    dataloader_mode="async_rebatched",
    dataloader_drop_last=True,
    dataloader_num_workers=64,
    logging_steps=5
)

**Note that we do not set evaluation to be performed during the training process for SQuAD**. This is due to the custom post-processing steps required to extract text-level answers for SQuAD, for which the logits cannot be easily modified without multiple function inputs, such as the tokenized and raw datasets, while the `preprocess_logits_for_metrics` argument provided in `IPUTrainingArguments` can only utilise logits alone. Therefore, validation is done after training.

We will need a data collator that will batch our processed samples together. Here we will use the default data collator imported from the [🤗 Transformers library](https://huggingface.co/docs/transformers/index). This is passed to the `IPUTrainer` class. 

Then we just need to pass all of this along with our datasets to the IPUTrainer:

In [ ]:
from transformers import default_data_collator

trainer = IPUTrainer(
    model=model,
    ipu_config=ipu_config,
    args=training_args,
    train_dataset=packed_train_dataset,
    data_collator=default_data_collator
)


We can now fine-tune our model by just calling the train method:

In [ ]:
train_run_metrics = trainer.train()

You can now upload the result of the training to the Hugging Face Hub if you successfully authenticated at the beginning of this notebook. Simply uncomment and execute the following cell:


In [ ]:
#trainer.push_to_hub()

Then save the model with the model checkpoint name.

In [ ]:
trainer.save_model(f"./{model_checkpoint}-{model_task}")

We can then perform the evaluation by using the `predict` method in `IPUTrainer`. This provides all of the raw predictions for the packed inputs for validation. This will, by default, use the global batch size defined specifically for inference in `IPUTrainingArguments`.

In [ ]:
raw_predictions = trainer.predict(packed_val_dataset)

Once the predictions have been obtained, the validation metrics can be computed by passing them into `compute_validation_metrics` function. This, as described previously, performs the necessary post-processing on the logits and obtains text answers. It then computes the accuracy metrics (exact match and F1 score) for SQuAD fine-tuning.

In [ ]:
val_metrics = compute_validation_metrics(
    raw_predictions.predictions, raw_validation_dataset, packed_val_dataset, metric)

print(val_metrics)

## Faster batched inference

Packing can also be used for inference, particularly for performing inference for workloads. This section demonstrates how to perform faster, batched inference with a large number of samples using a super-easy custom pipeline which batches and packs your input data, performs inference and returns post-processed predictions. 

For the pipeline, we need to import it, and initialise a few essential parameters.

The `model` is the model checkpoint, we are going to use the locally saved checkpoint generated from training SQuAD. Values for `executable_cache_dir` and `max_seq_length` must also be specified.

The pipeline will automatically determine your model's IPU config, given that the checkpoint was trained using Optimum Graphcore, which will be the case for the model fine-tuned in this notebook.

In this example, we pre-load the IPUConfig and modify some of the default parameters to get the best performance out of inference and leverage the benefits of IPU parallelism. The micro-batch size can also be specified, for which the default is 1.

When training, the packing factor affects the convergence the same way as a large increase in batch size would do. However, for inference, we are free to use a bigger packing factor to speed it up. Let's try it with `max_seq_per_pack = 12`.

**Note:** Packing brings huge benefits for performing inference on large amounts of data. For small scale inference tasks, such as those which more suit sequential inference on a single un-batched input, the generic Optimum Graphcore `TextClassificationPipeline` class may be preferred. This won't affect fine-tuning, and the weights generated from fine-tuning using packing will work just the same!

In [ ]:
from pipeline.packed_bert import PackedBertQuestionAnsweringPipeline

model = "./"+f"{model_checkpoint}-{model_task}"
# model = 'your_username/{model_name}-{model_task}' # to load from Hugging Face Hub

inference_boosted_ipu_config = IPUConfig.from_pretrained(model, 
        inference_device_iterations=32,
        inference_replication_factor=4,
        ipus_per_replica=1,
        layers_per_ipu=[12]
    )

pipeline = PackedBertQuestionAnsweringPipeline(
    model = f"./{model_checkpoint}-{model_task}",
    executable_cache_dir = executable_cache_dir,
    max_seq_per_pack=12,
    max_seq_length=max_seq_length,
    ipu_config=inference_boosted_ipu_config,
    micro_batch_size=8
)

The pipeline expects a **list of strings** directly passed to it in the format: 
```
questions=[<list of questions>], contexts=[<list of contexts>]
```
There is no need to tokenize, preprocess, pack or post-process the data to use the inference pipeline.

As a test, we can load the entire SQuAD validation dataset and perform packed inference using `.predict()` on the text column to generate predictions. Post-processing samples for SQuAD is done on a sample-by-sample, unbatched basis so this may take a few minutes with or without packing.

In [ ]:
import datasets
dataset = datasets.load_dataset('squad')
preds = pipeline.predict(questions=dataset['validation']['question'],
                         contexts=dataset['validation']['context'])

print(preds.keys())
print(f"Number of predictions: {len(preds['predictions'])}")
print(f"Preprocessing time: {preds['preprocessing_time']}s")
print(f"Postprocessing time: {preds['postprocessing_time']}s")
print(f"Throughput: {preds['throughput']} samples/s")

There is minimal overhead from tokenizing and packing the dataset, but the speed benefits for inference are evident. Running the above pipeline, we achieve a throughput approximately 6000 samples per second, showing an approximate 2x speed up for SQuAD.

## Next steps

You may want to try out the following notebooks using BERT with packing for:

* Single-label text classification - `packedBERT_single_label_text_classification.ipynb`
* Multi-label text classification - `packedBERT_multi_label_text_classification.ipynb`

Also, check out the full list of [IPU-powered Jupyter Notebooks](https://www.graphcore.ai/ipu-jupyter-notebooks) to get more of a feel for how IPUs perform on other tasks.